# RAG con 2 tools

In [2]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

### Creamos la conexión con Chroma y los objetos de los 2 embeddings

In [ ]:
# Ahora tenemos 2 bases de datos vectoriales con diferente información
# nutrition_db contiene información nutricional de alimentos específicos
# nutrition_qna contiene información nutricional de preguntas y respuestas (obtenidas de Kaggle)

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

### Creacion de las 'function_tool'

Una es la misma del ejemplo anterior.

La otra es una tool especializada en hacer busquedas en la vector store de nutricion.

In [4]:
# Creamos la función que se encargará de buscar la información en la base de datos de embeddings
# Es importante crear un docstring para la función que describa su comportamiento, datos de entrada y salida
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Function Tool para una base de datos RAG que busca información de calorías para alimentos específicos (pero no para comidas completas.

    Args:
        query: El alimento a buscar.
        max_results: El número máximo de resultados a devolver.

    Returns:
        Una cadena de texto conteniendo la información nutricional.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Formatear resultados para el agente
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Informacion nutricional:\n" + "\n".join(formatted_results)

In [5]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Función herramienta para hacer preguntas sobre nutrición.

    Args:
        query: La pregunta a realizar
        max_results: El número máximo de resultados a devolver.

    Returns:
        Una cadena de texto conteniendo la pregunta y la respuesta relacionada con la consulta.
    """

    results = nutrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Formatear resultados para el agente
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Respuestas relacionadas con tu pregunta:\n" + "\n".join(formatted_results)

### Creamos el agente

En este caso hacemos referencia a las 2 tools en sus argumentos.

In [14]:
# Creamos el agente ahora con 2 tools
calorie_agent = Agent(
    name="Asistente Nutricional con 2 RAGs",
    instructions="""
    Eres un asistente nutricional que proporciona información sobre calorías y consejos nutricionales.
    Das respuestas concisas.
    
    Si necesitas buscar información de calorías, usa el calorie_lookup_tool y no uses el nutrition_qna_tool.

    Si te hacen una pregunta sobre nutrición, siempre usa primero el nutrition_qna_tool para ver si hay una respuesta en la base de conocimiento.

    Indica siempre con una etiqueta si la respuesta ha usado alguna de las tools asociadas al agente, por ejemplo 'basado en nutrition_qna_tool' o 'basado en calorie_lookup_tool'
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [10]:
with trace("Asistente Nutricional con RAG de Nutrición y Calorías"):
    result = await Runner.run(
        calorie_agent,
        "Cuáles son las mejores opciones alimenticias para las mujeres embarazadas y cuántas calorías tienen?",
    )
    print(result.final_output)

basado en nutrition_qna_tool

Opciones alimentarias recomendadas durante el embarazo (con calorías aproximadas por porción cuando aplica):
- Frutas y verduras variadas: fuente de vitaminas, fibra y folato. Calorías: variable, foco en 5-7 porciones/día.
- Cereales integrales y legumbres: avena, quinoa, arroz integral, lentejas; buen aporte de energía y fibra. Calorías por porción ~150-200.
- Proteínas magras: pollo, pescado bajo en mercurio, huevos, tofu, legumbres; clave para crecimiento fetal. Calorías por porción ~100-180 (según fuente).
- Lácteos o equivalents: leche, yogurt, queso; aporte de calcio y proteína. Calorías por porción ~80-150.
- Grasas saludables: aceite de oliva, aguacate, frutos secos; DHA/ángeles para desarrollo cerebral. Calorías por porción ~100-200 (depende del alimento).
- Alimentos ricos en ácido fólico, hierro y calcio: espinaca, legumbres, carnes magras, cárnicos fortificados, leche/yogur fortificados.
- Pescados con omega-3 y bajo mercurio: salmón, sardinas;

In [15]:
with trace("Asistente Nutricional con RAG de Nutrición y Calorías"):
    result = await Runner.run(
        calorie_agent,
        "Cuantas calorias tiene el platano y la naranja si los comemos juntos (50g + 50g)?",
    )
    print(result.final_output)

Total aproximado: 68 kcal.

- Banana 50 g ≈ 44.5 kcal
- Naranja 50 g ≈ 23.5 kcal

basado en calorie_lookup_tool


In [16]:
with trace("Asistente Nutricional con RAG de Nutrición y Calorías"):
    result = await Runner.run(
        calorie_agent,
        "En qué consiste la Dieta Mediterranea?",
    )
    print(result.final_output)

basado en nutrition_qna_tool

La Dieta Mediterránea se basa en:
- Frutas, verduras y granos enteros
- Legumbres, frutos secos y aceite de oliva como grasa principal
- Pescado y mariscos varias veces a la semana; pollo y huevos con moderación
- Consumo moderado de lácteos; cantidades pequeñas de queso y yogur
- Consumo limitado de carnes rojas y ultraprocesados
- Hierbas, especias y uvas o vino con moderación en comidas
- Comer en familia, compartir y preferir alimentos de temporada
- Estilo de vida activo y evita procesados

Beneficios: salud cardiaca, control de peso y reducción de inflamación.
